In [1]:
SENDER_NAME = "狩野"
SENDER_ROMANAME = "kano"
# シート名
SHEET_NAME: str = 'SendList'
# urlのhttps://docs.google.com/spreadsheets/d/xxxxxxxxxxxxxxxxxxxxxxx/edit#gid=1895778341 xxxxxの部分
SPREADSHEET_KEY: str = '1pZez0B9FZyzw64nSeqRYkeq6o60p9IojlTG5V_Va8po'

# 件名の設定
SUBJECT = "【スペシャルオファー】"

def generateMessage(
    SENDER_NAME: str, 
    TALENT_NAME: str, ):

    message: str = f"""{TALENT_NAME}さん、

    突然のご連絡失礼します。
    ハイタレント株式会社の{SENDER_NAME}と申します。

    弊社では高単価のコンサル案件を中心とした副業・フリーランス案件マッチングサービスを運営しております。
    LinkedInでご経歴をお見かけし、{TALENT_NAME}さんにハイタレントをご活用いただけると思い、ご連絡差し上げました。

    ご不明点ございましたら、何なりとお申し付けください。 
    どうぞよろしくお願いいたします。

    """
    return message


In [2]:
import os
import time
from pathlib import Path
import subprocess
import re

import gspread
import json
import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoAlertPresentException
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager


from dotenv import load_dotenv

from lib.utils import getSpreadSheet

## Spread Sheetのデータをfetch

# 送信するlinkedinアカウント名
SPREADSHEET_ACCOUNT_NAME_COLUMN: int = 5
# 送信するlinkedinアカウントURL
SPREADSHEET_ACCOUNT_COLUMN: int = 3
# 送信対象フラグ
# SPREADSHEET_IS_TARGET: str = '18'
# 送信フラグ
SPREADSHEET_IS_SENT: int = 8
# 送信日時: AP列みたいな書き方
SPREADSHEET_SENT_DATE_ENG: str = 'F'
# 送信者を入力する列
SPREADSHEET_SENDER : str = 'G'

# SPREADSHEET_ABTEST : str = '19'

# 過去送付フラグ列
# SPREADSHEET_PAST_SEND: str = '22'
# 送信したメッセージのパターン；AP列みないな書き方
# SPREADSHEET_MESSAGE_PATTERN: str = "21"

# GLOBAL変数
## LinkedinのCredentialをロード
load_dotenv(f"config./.{SENDER_ROMANAME}.env")

USER_NAME = os.environ.get("USER_NAME")
PASSWORD = os.environ.get("PASSWORD")
OAUTH = os.environ.get("OAUTH")


chrome_options = Options()
# その他のオプション設定が必要な場合、ここに追加...
try:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
except:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

driver.get('https://www.linkedin.com/login/ja?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

# usernameのinput要素を指定
username = driver.find_element(By.ID,'username')
time.sleep(1)

# usernameのinput要素にログインid(email)を入力
username.send_keys(USER_NAME)
time.sleep(1)

# passwordのinput要素を指定
password = driver.find_element(By.ID,'password')
time.sleep(1)

# passwordのinput要素にパスワードを入力
password.send_keys(PASSWORD)
time.sleep(1)

# ログインボタンを指定
log_in_button = driver.find_element(By.XPATH,'//*[@type="submit"]')
time.sleep(1)

# ログインボタンをクリック
log_in_button.click()
time.sleep(1)

In [3]:
# INMAILを送るリストをSpread sheetから作成
worksheet = getSpreadSheet(SPREADSHEET_KEY, SHEET_NAME)
nameColumn = worksheet.col_values(SPREADSHEET_ACCOUNT_NAME_COLUMN)
urlColumn = worksheet.col_values(SPREADSHEET_ACCOUNT_COLUMN)
# isTargetColumn = worksheet.col_values(SPREADSHEET_IS_TARGET)
sendStatus= worksheet.col_values(SPREADSHEET_IS_SENT)

def insertSpreadSheet(worksheet, column: str, row: str, text: str):
    worksheet.update_acell(f'{column}{row}', text)

t_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(t_delta, 'JST')
print(len(urlColumn))
for i in range(len(urlColumn)):
    if i <= 0:
        continue
    # 送付対象からむをチェックして0だったら対象外
    # elif isTargetColumn[i] == "0":
    #     print(f"pass: {i} {nameColumn[i]}")
    #     continue
    # 既に送付済みだったら送付しない
    # elif isSentColumn[i] != "":
    #     print(f"pass: {i} {nameColumn[i]}")
    #     continue
    # elif sendStatus[i] == 1:
    #     continue
    else:
        try:
            # 新しいタブを開く
            driver.execute_script("window.open('');")
            # フォーカスを新しいタブに移動
            handle_array = driver.window_handles
            driver.switch_to.window(handle_array[-1])

            url = urlColumn[i]
            driver.get(url)

            time.sleep(10)

            # try:
            #         ConnectButton = driver.find_elements(By.XPATH,"//div[@class='pvs-profile-actions']//button/li-icon[contains(@type, 'connect')]")
            #         if len(ConnectButton) > 0:
            #             driver.execute_script("arguments[0].click();", ConnectButton[0])

            #         else:
            #             # Moreボタンを探してクリック
            #             MoreButton = driver.find_elements(By.XPATH,"//button[@aria-label='その他のアクション']")
            #             driver.execute_script("arguments[0].click();", MoreButton[0])
            #             time.sleep(2)

            #             # Connentボタンを探してクリック
            #             ConnectButton = driver.find_elements(By.XPATH,"//div[contains(@class, 'pvs-profile-actions')]//li-icon[contains(@type, 'connect')]")
            #             driver.execute_script("arguments[0].click();", ConnectButton[0])
            #             time.sleep(2)

            #         # Sendボタンをクリック->繋がり申請が送られるので、次のタレントに移動する
            #         # Add a noteボタンを探してクリック
            #         SendButton = driver.find_elements(By.XPATH,"//button[@aria-label='今すぐ送信']")
            #         if len(SendButton) == 0:
            #             print('送信ボタンが見つかりませんでした')
            #         time.sleep(2)
            #         driver.execute_script("arguments[0].click();", SendButton[0])

            #         # 繋がり申請の上限を検知
            #         AlertArea = driver.find_elements(By.XPATH,"//div[@aria-labelledby='ip-fuse-limit-alert__header']")
            #         if len(AlertArea)>0:
            #             print("connect limitation: ", url)
            #             raise Exception
            # except:
            #     print("connect limitation: ", url)
            #     raise Exception


            # time.sleep(5)
            # Sales Navigatorボタンを探す
            # try:
            #     SalesNavigatorButton = driver.find_elements(By.XPATH,"//div[contains(@class, 'pvs-profile-actions')]/button[contains(@aria-label, 'Sales Navigatorで表示')]")
            #     SalesNavigatorButtonInOthers = driver.find_elements(By.XPATH,"//span[contains(text(), 'Sales Navigatorで表示')]")
            #     if len(SalesNavigatorButton) > 0:
            #         driver.execute_script("arguments[0].click();", SalesNavigatorButton[0])
                
            #     elif len(SalesNavigatorButtonInOthers) > 0:
            #         driver.execute_script("arguments[0].click();", SalesNavigatorButtonInOthers[0])
            #     else:
            #         raise Exception("Sales Navigatorで表示ボタンが見つかりませんでした")
            # except:
            #     print('Sales Navigatorで表示ボタンが見つかりませんでした')
            #     raise Exception("Sales Navigatorで表示ボタンが見つかりませんでした")

            # # 新規タブで開かれるので、フォーカスを移動
            # time.sleep(3)
            # handle_array = driver.window_handles
            # driver.switch_to.window(handle_array[-1])

            try:
                messageButton = driver.find_elements(By.XPATH,"//button[@data-anchor-send-inmail]")
                driver.execute_script("arguments[0].click();", messageButton[0])
                time.sleep(5)
            except:
                print('Messageボタンが見つかりませんでした')

            try:
                # メールタイトルフォーム
                subjectForm = driver.find_elements(By.XPATH,"//input[contains(@id, 'compose-form-subject')]")
                # メールタイトルAフォーム
                subjectForm[0].send_keys(SUBJECT)
                # メール本文Aフォーム
                contentForm = driver.find_elements(By.XPATH,"//textarea[contains(@name, 'message')]")
                contentForm[0].send_keys(generateMessage(
                    SENDER_NAME=SENDER_NAME,
                    TALENT_NAME=nameColumn[i],
                    ))
                print(generateMessage(
                    SENDER_NAME=SENDER_NAME,
                    TALENT_NAME=nameColumn[i],
                    ))
            
            except:
                print('タイトル/本文入力エラー')
                print(generateMessage(
                    SENDER_NAME=SENDER_NAME,
                    TALENT_NAME=nameColumn[i],
                    ))

            # try:
            #     # 添付ファイルフォーム
            #     attachmentForm = driver.find_elements(By.XPATH,"//input[@name='file']")
            #     attachmentForm[0].send_keys(f"{Path().resolve()}/{ATTACHMENT_FILE}")
            #     time.sleep(5)
            # except:
            #     print('添付ファイルエラー')

            try:
                # 送信ボタンをクリック
                sendButton = driver.find_elements(By.XPATH,"//span[@tabindex='-1']/button/span")
                # driver.execute_script("arguments[0].click();", sendButton[0])
                insertSpreadSheet(worksheet=worksheet, column=SPREADSHEET_SENT_DATE_ENG, row=i+1, text=datetime.datetime.now(JST).strftime('%Y/%m/%d'))
                insertSpreadSheet(worksheet=worksheet, column=SPREADSHEET_SENDER, row=i+1, text=SENDER_NAME)
                insertSpreadSheet(worksheet=worksheet, column=SPREADSHEET_IS_SENT, row=i+1, text=1)
                # insertSpreadSheet(worksheet=worksheet, column=SPREADSHEET_MESSAGE_PATTERN, row=i+1, text=message_pattern)
            except:
                print('Sendボタンが見つかりませんでした')
            
            # # タブを閉じる
            # タブを閉じる前にメインウィンドウのハンドルを保存
            main_window = driver.window_handles[0]
            
            # 現在のタブを閉じる
            driver.close()
            time.sleep(1)
            
            # メインウィンドウにフォーカスを戻す
            driver.switch_to.window(main_window)
            
            # アラートが表示された場合の処理
            try:
                alert = driver.switch_to.alert
                alert.accept()
            except NoAlertPresentException:
                pass

            # driver.close()
            # time.sleep(1)
            # alert = driver.switch_to_alert()
            # alert.accept()
        except Exception as e:
            print(f'error: {e}')

3
タイトル/本文入力エラー
Sendボタンが見つかりませんでした
タイトル/本文入力エラー
Sendボタンが見つかりませんでした


In [4]:
# GLOBAL変数
## LinkedinのCredentialをロード
load_dotenv(f"config./.{SENDER_ROMANAME}.env")


USER_NAME = os.environ.get("USER_NAME")
PASSWORD = os.environ.get("PASSWORD")
OAUTH = os.environ.get("OAUTH")
print(USER_NAME)
print(PASSWORD)
# print(OAUTH)


tadanosinkano@icloud.com
tibisuke0729
